In [1]:
%load_ext autoreload
%autoreload 2
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from lark import Lark
import time 
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

# prepare dataset

In [14]:
def sklearn_to_df(sklearn_dataset):
    """ 
    Convert sklearn dataset to a dataframe, the class-label is renamed to "target"
    """
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df


target = "target"
dataset = datasets.load_iris()
dataset[target] = np.where(dataset[target]==2, 0, dataset[target])



# get df
data = sklearn_to_df(dataset)

# get X,y
X = data.drop(['target'], axis=1)
y = data['target']

scaler = MinMaxScaler()
X[X.columns] = scaler.fit_transform(X[X.columns])



X.columns = ['sepal length', 'sepal width', 'petal length', 'petal width']


# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # 70% training and 30% test


X_train

,sepal length,sepal width,petal length,petal width
69,0.361111,0.208333,0.491525,0.416667
147,0.611111,0.416667,0.711864,0.791667
124,0.666667,0.541667,0.796610,0.833333
31,0.305556,0.583333,0.084746,0.125000
37,0.166667,0.666667,0.067797,0.000000
...,...,...,...,...
14,0.416667,0.833333,0.033898,0.041667
67,0.416667,0.291667,0.525424,0.375000
11,0.138889,0.583333,0.101695,0.041667
113,0.388889,0.208333,0.677966,0.791667


# train a black-box

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

clf_rf=RandomForestClassifier(n_estimators=100)
clf_mlp = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)

clf_rf.fit(X_train,y_train)
clf_mlp.fit(X_train,y_train)

y_pred=clf_mlp.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


print(clf_rf.predict([[3, 5, 4, 2]]))
print(clf_mlp.predict([[3, 5, 4, 2]]))

Accuracy: 0.9333333333333333
[0]
[0]


# create a random generator

In [21]:

import random
def random_generator(X):
    num_attributes = len(X.columns)
    x=[]
    for i in range(num_attributes):
        x.append(random.uniform(X[X.columns[i]].min(),X[X.columns[i]].max()))
    return x

for i in range(5):
    x = random_generator(X)
    print(x)
    print(clf_rf.predict([x]))
    print(clf_mlp.predict([x]))

[0.7601654389594489, 0.5355927947452576, 0.4425778617102335, 0.2847039548109074]
[1]
[1]
[0.8023529688291837, 0.9490040871917887, 0.4839349153782212, 0.28534969360189055]
[1]
[0]
[0.23103259815382415, 0.7473302538299597, 0.21230897410779437, 0.7107714587541916]
[0]
[0]
[0.09781158536621726, 0.7533534524398341, 0.7729278794495378, 0.5866817155086415]
[0]
[0]
[0.11492718002106783, 0.6786092051572782, 0.6842296933638288, 0.687392542189933]
[0]
[0]


# implementation of learner

### Synthetic dataset

In [22]:
from sygus_if import SyGuS_IF
X = [
    [1,2,1,1],
    [1,1,1,42]
]
y = [
    1,
    0
]
sgf = SyGuS_IF(feature_data_type={0: "Real", 1: "Real", 2: "Real", 3: "Real"}, function_return_type="Bool")
sgf.fit(X,y)
print(sgf.synthesized_function)

[[1, 2, 1, 1], [1, 1, 1, 42]] [1, 0]
(define-fun func ((x_0 Real) (x_1 Real) (x_2 Real) (x_3 Real)) Bool (> x_1 1.0))


## iris dataset

In [33]:
from sygus_if import SyGuS_IF
sgf = SyGuS_IF(feature_data_type=None)
sgf.fit(X_train, y_train)
print(sgf.synthesized_function)

CalledProcessError: Command 'cvc4 --lang=sygus2 input.sl' returned non-zero exit status 1.

In [6]:
# start_ = time.time()



# y_pred_test = sgf.predict_z3(X_test)
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
# y_pred_train = sgf.predict_z3(X_train)
# print("Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
# print(y_pred_test)
# print(time.time() - start_)

In [7]:


# start_ = time.time()

# y_pred_test = sgf.predict(X_test, y_test)
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
# y_pred_train = sgf.predict(X_train, y_train)
# print("Accuracy:",metrics.accuracy_score(y_train, y_pred_train))

# print(time.time() - start_)

# implementation of the query

In [8]:
import query
import numpy as np
def predict_function_query(x):
    a = np.array([0.5, 0.5, 0.5, 0.5])
    dist = np.linalg.norm(a-np.array(x))
    if(dist < 0.5):
        return True
    else:
        return False

q = query.Query(model = None, prediction_function = predict_function_query)
q.classify_example([1, 0.5, 0.5, 0.55])

False

# implementation of black-box

In [9]:
from blackbox import BlackBox
bb = BlackBox(clf_mlp, clf_mlp.predict)
bb.classify_example([3, 5, 4, 2])

False

# implementation of teacher

In [10]:
from teacher import Teacher
from learner import Learner
sgf = SyGuS_IF()
l = Learner(model = sgf, prediction_function = sgf.predict_z3, train_function = sgf.fit, X = [], y=[] )

t = Teacher(max_iterations=3,epsilon=0.03, delta=0.03)
new_l, flag = t.teach(blackbox = bb, learner = l, query = q, random_example_generator = random_generator, params_generator = X)
print()
print("\nLearning complete")
print("Is learning complete?", flag)
print(new_l.model.synthesized_function)
print(new_l.y)

0%|          | 0/3 [00:00<?, ?it/s]


TypeError: 'NoneType' object does not support item assignment

In [56]:
start_ = time.time()

cnt = 0
for example in X.values.tolist():

    blackbox_verdict = bb.classify_example(example)
    learner_verdict = new_l.classify_example(example)
    query_verdict = q.classify_example(example)

    if(learner_verdict == (blackbox_verdict and query_verdict)):
        cnt += 1
print("correct: ", cnt, "out of ", len(y), "examples. Percentage: ", cnt/len(y))
print(time.time() - start_)

correct:  119 out of  150 examples. Percentage:  0.7933333333333333
3.652432680130005


## Learning on Zoo dataset using SyGus

In [21]:
from data.objects import zoo
from sygus_if import SyGuS_IF

dataObj = zoo.Zoo()
df = dataObj.get_df()

# fix target class
target_class = 2 
_temp = {}
for i in range(1, len(df[dataObj.target].unique())+1):
    if(i == target_class):
        _temp[i] = 1
    else:
        _temp[i] = 0
print(_temp)
df[dataObj.target] = df[dataObj.target].map(_temp)
df.head()

# declaration of classifier, X and y
sgf = SyGuS_IF(feature_data_type=dataObj.attribute_type, function_return_type= "Bool")
X = df.drop([dataObj.target], axis=1)
y = df[dataObj.target]

print(y.tolist())
# train
sgf.fit(X,y)
print(sgf.synthesized_function)

-number of samples: (before dropping nan rows) 101
-number of samples: (after dropping nan rows) 101
{1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
[[1. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 1. 0. 1.]
 [0. 0. 1. ... 1. 0. 0.]
 ...
 [1. 0. 0. ... 1. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 1. 0. 0.]] 0      0
1      0
2      0
3      0
4      0
      ..
96     0
97     0
98     0
99     0
100    1
Name: class_type, Length: 101, dtype: int64
(define-fun func ((hair Bool) (feathers Bool) (eggs Bool) (milk Bool) (airborne Bool) (aquatic Bool) (predator Bool) (toothed Bool) (backbone Bool) (breathes Bool) (venomous Bool) (fins Bool) (legs Real) (tail Bool) (domestic Bool) (catsize Bool)) Bool 

In [36]:
start_ = time.time()
y_pred_test = sgf.predict_z3(X)
print("Accuracy:",metrics.accuracy_score(y, y_pred_test))
print(time.time() - start_)

Accuracy: 1.0
1.9228363037109375


In [37]:
start_ = time.time()



y_pred_test = sgf.predict(X, y)
print("Accuracy:",metrics.accuracy_score(y, y_pred_test))
print(time.time() - start_)

Accuracy: 1.0
2.5174355506896973
